<a href="https://colab.research.google.com/github/Daffaaq/Prediction-MPL-ID-S16-metode-Logistic-Regression/blob/main/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# ================================================
# 1. Hardcoded Match Data (Week 1 & 2)
# ================================================

match_results = [
    # Week 1
    {"team1": "ONIC", "team2": "Dewa", "score1": 2, "score2": 0},
    {"team1": "NAVI", "team2": "EVOS", "score1": 0, "score2": 2},
    {"team1": "TLID", "team2": "GEEK", "score1": 1, "score2": 2},
    {"team1": "ONIC", "team2": "BTR", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "AE", "score1": 2, "score2": 1},
    {"team1": "BTR", "team2": "NAVI", "score1": 1, "score2": 2},
    {"team1": "GEEK", "team2": "RRQ", "score1": 2, "score2": 0},
    {"team1": "AE", "team2": "Dewa", "score1": 2, "score2": 0},

    # Week 2
    {"team1": "ONIC", "team2": "GEEK", "score1": 2, "score2": 0},
    {"team1": "Dewa", "team2": "NAVI", "score1": 2, "score2": 0},
    {"team1": "GEEK", "team2": "BTR", "score1": 0, "score2": 2},
    {"team1": "AE", "team2": "EVOS", "score1": 2, "score2": 1},
    {"team1": "TLID", "team2": "Dewa", "score1": 1, "score2": 2},
    {"team1": "NAVI", "team2": "AE", "score1": 2, "score2": 0},
    {"team1": "RRQ", "team2": "TLID", "score1": 2, "score2": 0},
    {"team1": "BTR", "team2": "EVOS", "score1": 2, "score2": 1},
]


# ================================================
# 2. Generate Statistik Tiap Tim
# ================================================

teams = set()
stats = {}

# Initialize stats
for match in match_results:
    teams.update([match["team1"], match["team2"]])
for team in teams:
    stats[team] = {
        "match_played": 0,
        "match_won": 0,
        "game_won": 0,
        "game_lost": 0,
        "last_results": []  # win/loss streak
    }

# Simulasikan setiap match sebagai skor 2-0 atau 2-1 (acak jika menang)
import random
random.seed(42)

for match in match_results:
    t1, t2 = match["team1"], match["team2"]
    score1, score2 = match["score1"], match["score2"]

    stats[t1]["match_played"] += 1
    stats[t2]["match_played"] += 1

    stats[t1]["game_won"] += score1
    stats[t1]["game_lost"] += score2

    stats[t2]["game_won"] += score2
    stats[t2]["game_lost"] += score1

    if score1 > score2:
        stats[t1]["match_won"] += 1
        stats[t1]["last_results"].append("W")
        stats[t2]["last_results"].append("L")
    else:
        stats[t2]["match_won"] += 1
        stats[t2]["last_results"].append("W")
        stats[t1]["last_results"].append("L")


# ================================================
# 3. Encode fitur: Tim1 - Tim2 jadi vector fitur
# ================================================

def build_feature(team1, team2):
    def get_team_vector(team):
        s = stats[team]
        match_wr = s["match_won"] / s["match_played"] if s["match_played"] else 0
        game_wr = s["game_won"] / (s["game_won"] + s["game_lost"]) if (s["game_won"] + s["game_lost"]) else 0
        streak = sum([1 if r == "W" else -1 for r in s["last_results"][-3:]])  # last 3 games
        return [match_wr, game_wr, streak]

    vec1 = get_team_vector(team1)
    vec2 = get_team_vector(team2)

    # Bisa gunakan perbedaan sebagai fitur
    diff = [a - b for a, b in zip(vec1, vec2)]
    return diff

X = []
y = []

for match in match_results:
    x = build_feature(match["team1"], match["team2"])
    winner = match["team1"] if match["score1"] > match["score2"] else match["team2"]
    label = 1 if winner == match["team1"] else 0
    X.append(x)
    y.append(label)


# ================================================
# 4. Train Model Logistic Regression
# ================================================

print("=== Hasil Match Week 1 & 2 ===")
for match in match_results:
    print(f"{match['team1']} {match['score1']}:{match['score2']} {match['team2']}")

standings_before = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    standings_before.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

standings_before.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

print("\n=== Klasemen Setelah Week 1 & 2 ===")
print(f"{'Pos':<4} {'Team':<10} {'W':<2} {'L':<2} {'GW':<3} {'GL':<3} {'GD':<3}")
for i, s in enumerate(standings_before, 1):
    print(f"{i:<4} {s['team']:<10} {s['match_won']:<2} {s['match_played'] - s['match_won']:<2} "
          f"{s['game_won']:<3} {s['game_lost']:<3} {s['game_diff']:<3}")


model = LogisticRegression()
model.fit(X, y)

# ================================================
# 5. Prediksi Week 3
# ================================================

week3_matches = [
    {"team1": "NAVI", "team2": "RRQ"},
    {"team1": "EVOS", "team2": "TLID"},
    {"team1": "EVOS", "team2": "GEEK"},
    {"team1": "BTR", "team2": "AE"},
    {"team1": "RRQ", "team2": "ONIC"},
    {"team1": "Dewa", "team2": "BTR"},
    {"team1": "AE", "team2": "ONIC"},
    {"team1": "TLID", "team2": "NAVI"},
]

# ================================================
# 6. Update Statistik Berdasarkan Prediksi Week 3
# ================================================
def simulate_match(team1, team2, predicted_winner):
    loser = team2 if predicted_winner == team1 else team1
    win_score = 2
    lose_score = random.choice([0, 1])  # Simulasi skor menang (bisa 2-0 atau 2-1)

    # Update statistik
    stats[team1]["match_played"] += 1
    stats[team2]["match_played"] += 1

    stats[predicted_winner]["match_won"] += 1
    stats[predicted_winner]["game_won"] += win_score
    stats[predicted_winner]["game_lost"] += lose_score

    stats[loser]["game_won"] += lose_score
    stats[loser]["game_lost"] += win_score

    stats[predicted_winner]["last_results"].append("W")
    stats[loser]["last_results"].append("L")

    # Kembalikan skor untuk dicetak
    return (win_score, lose_score) if predicted_winner == team1 else (lose_score, win_score)



print("\n=== Prediksi Week 3 ===")
for match in week3_matches:
    x_pred = [build_feature(match["team1"], match["team2"])]
    prob = model.predict_proba(x_pred)[0][1]  # Prob team1 menang
    pred_winner = match["team1"] if prob >= 0.5 else match["team2"]
    score1, score2 = simulate_match(match["team1"], match["team2"], pred_winner)
    print(f"{match['team1']} {score1}:{score2} {match['team2']} => Prediksi: {pred_winner} menang (Prob: {prob:.2f})")


# ================================================
# 7. Buat Klasemen Setelah Week 3
# ================================================
standings = []
for team, s in stats.items():
    game_diff = s["game_won"] - s["game_lost"]
    standings.append({
        "team": team,
        "match_won": s["match_won"],
        "match_played": s["match_played"],
        "game_won": s["game_won"],
        "game_lost": s["game_lost"],
        "game_diff": game_diff,
    })

# Urutkan berdasarkan aturan MPL:
# 1. Match win paling banyak
# 2. Game difference
# 3. Game won (jika masih seri)
standings.sort(key=lambda x: (-x["match_won"], -x["game_diff"], -x["game_won"]))

# Tampilkan klasemen
print("\n=== Klasemen Akhir Week 3 (Prediksi) ===")
print(f"{'Pos':<4} {'Team':<10} {'W':<2} {'L':<2} {'GW':<3} {'GL':<3} {'GD':<3}")
for i, s in enumerate(standings, 1):
    print(f"{i:<4} {s['team']:<10} {s['match_won']:<2} {s['match_played'] - s['match_won']:<2} "
          f"{s['game_won']:<3} {s['game_lost']:<3} {s['game_diff']:<3}")



=== Hasil Match Week 1 & 2 ===
ONIC 2:0 Dewa
NAVI 0:2 EVOS
TLID 1:2 GEEK
ONIC 2:0 BTR
RRQ 2:1 AE
BTR 1:2 NAVI
GEEK 2:0 RRQ
AE 2:0 Dewa
ONIC 2:0 GEEK
Dewa 2:0 NAVI
GEEK 0:2 BTR
AE 2:1 EVOS
TLID 1:2 Dewa
NAVI 2:0 AE
RRQ 2:0 TLID
BTR 2:1 EVOS

=== Klasemen Setelah Week 1 & 2 ===
Pos  Team       W  L  GW  GL  GD 
1    ONIC       3  0  6   0   6  
2    RRQ        2  1  4   3   1  
3    BTR        2  2  5   5   0  
4    AE         2  2  5   5   0  
5    GEEK       2  2  4   5   -1 
6    Dewa       2  2  4   5   -1 
7    NAVI       2  2  4   5   -1 
8    EVOS       1  2  4   4   0  
9    TLID       0  3  2   6   -4 

=== Prediksi Week 3 ===
NAVI 2:0 RRQ => Prediksi: NAVI menang (Prob: 0.66)
EVOS 2:0 TLID => Prediksi: EVOS menang (Prob: 0.88)
EVOS 2:1 GEEK => Prediksi: EVOS menang (Prob: 0.68)
BTR 2:0 AE => Prediksi: BTR menang (Prob: 0.67)
RRQ 0:2 ONIC => Prediksi: ONIC menang (Prob: 0.15)
Dewa 0:2 BTR => Prediksi: BTR menang (Prob: 0.39)
AE 0:2 ONIC => Prediksi: ONIC menang (Prob: 0.15)
TLID